In [ ]:
import requests
import pandas as pd
import time
from google.cloud import bigquery
import datetime
import os


In [ ]:
# URL de l'API CoinGecko
API_URL = "https://api.coingecko.com/api/v3/coins/markets"

# Paramètres de la requête
params = {
    "vs_currency": "usd",  # Devise (USD, EUR, BTC, etc.)
    "order": "market_cap_desc",  # Trier par capitalisation boursière
    "per_page": 10,  # Nombre de cryptos à récupérer
    "page": 1,
    "sparkline": False  # Pas besoin des données de tendance
}

def fetch_crypto_data():
    """Récupère les données des cryptos depuis l'API CoinGecko"""
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data)  # Convertir en DataFrame
    else:
        print(f"Erreur {response.status_code} : {response.text}")
        return None

# Exécuter la requête et afficher les résultats
crypto_df = fetch_crypto_data()
print(crypto_df.columns)
df = crypto_df
# Transformation des données (ex. Conversion devise, moyennes mobiles)
df["timestamp"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
conversion_rate = 0.95  # Exemple de taux de change
df["current_price_eur"] = df["current_price"] * conversion_rate
df["price_7d_avg"] = df["current_price"].rolling(window=7).mean()
df["price_30d_avg"] = df["current_price"].rolling(window=30).mean()
df["price_change_percentage_24h_flag"] = df["price_change_percentage_24h"].apply(lambda x: "Anomalie" if abs(x) > 10 else "Normal")

print(crypto_df[["id", "symbol", "current_price", "market_cap", "price_change_percentage_24h", "timestamp", "price_7d_avg", "price_30d_avg", "current_price_eur", "price_change_percentage_24h_flag", "total_volume"]])

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')
             id symbol  current_price     market_cap  \
0       bitcoin    btc   97524.000000  1933254001929   
1      ethereum    eth    2684.670000   323539950393   
2        ripple    xrp       2.730000   157647955427   
3        tether   usdt       1.000000   142013103902   
4   binancecoin    bnb     658.840000    96120240333   
5        solana    sol     193.110000    94298956002   
6      usd-coin   usdc       0.999949    56350887912   
7      dogecoin   doge       0.269556    39875219

In [ ]:
import pandas as pd

def transform_crypto_data(df):
    """Transformation des données des cryptomonnaies."""

    # 1. Conversion des devises (par exemple, de USD à EUR)
    # On va simplement multiplier par un taux de conversion fictif, mais tu peux intégrer une API pour cela.
    conversion_rate = 0.95  # Exemple de taux de change USD -> EUR
    df["current_price_eur"] = df["current_price"] * conversion_rate


    # 3. Détection des anomalies : Si la variation est supérieure à 10%
    df["price_change_percentage_24h_flag"] = df["price_change_percentage_24h"].apply(lambda x: "Anomalie" if abs(x) > 10 else "Normal")

    return df

# Convertir en DataFrame
crypto_df = crypto_df[["id", "symbol", "current_price", "market_cap", "price_change_percentage_24h","timestamp", "price_7d_avg", "price_30d_avg","total_volume"]]
crypto_df["timestamp"] = pd.to_datetime(crypto_df['timestamp'])
# Appliquer la transformation
transformed_df = transform_crypto_data(crypto_df)

# Afficher les résultats
print(transformed_df)

             id symbol  current_price     market_cap  \
0       bitcoin    btc   97524.000000  1933254001929   
1      ethereum    eth    2684.670000   323539950393   
2        ripple    xrp       2.730000   157647955427   
3        tether   usdt       1.000000   142013103902   
4   binancecoin    bnb     658.840000    96120240333   
5        solana    sol     193.110000    94298956002   
6      usd-coin   usdc       0.999949    56350887912   
7      dogecoin   doge       0.269556    39875219514   
8       cardano    ada       0.778534    27941754583   
9  staked-ether  steth    2684.440000    25217348081   

   price_change_percentage_24h           timestamp  price_7d_avg  \
0                      0.43232 2025-02-15 21:02:19           NaN   
1                     -1.47350 2025-02-15 21:02:19           NaN   
2                     -0.40736 2025-02-15 21:02:19           NaN   
3                     -0.01975 2025-02-15 21:02:19           NaN   
4                     -0.37337 2025-02-15 2

<ipython-input-36-c409777eb759>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crypto_df["timestamp"] = pd.to_datetime(crypto_df['timestamp'])


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Variables d'API et BigQuery
PROJECT_ID = os.getenv("scenic-comfort-450700-h8")
DATASET_ID = "crypto_data"
TABLE_ID = "crypto_data_1"

os.environ["GOOGLE_CLOUD_PROJECT"] = "scenic-comfort-450700-h8"


def fun_enregistrement(df, PROJECT_ID, DATASET_ID, TABLE_ID):

  # Enregistrement dans BigQuery
    client = bigquery.Client()
    table_ref = client.dataset(DATASET_ID).table(TABLE_ID)
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
    load_job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

    load_job.result()  # Attendre que l'import soit terminé

fun_enregistrement(transformed_df, PROJECT_ID, DATASET_ID, TABLE_ID)